In [1]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords

C:\Users\sriram\Anaconda3\lib\site-packages\gensim-1.0.1-py3.5-win-amd64.egg\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2017-06-18 18:40:12,780 : INFO : 'pattern' package not found; tag filters are not available for English
C:\Users\sriram\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
%%time 
wv = gensim.models.KeyedVectors.load_word2vec_format(
"C:\\Users\\sriram\\Downloads\\GoogleNews-vectors-negative300.bin.gz",
binary=True)
wv.init_sims(replace=True)

2017-06-18 18:40:58,854 : INFO : loading projection weights from C:\Users\sriram\Downloads\GoogleNews-vectors-negative300.bin.gz
2017-06-18 18:43:28,944 : INFO : loaded (3000000, 300) matrix from C:\Users\sriram\Downloads\GoogleNews-vectors-negative300.bin.gz
2017-06-18 18:43:28,944 : INFO : precomputing L2-norms of word weight vectors


Wall time: 2min 53s


In [3]:
path="C:\\Users\\sriram\\OneDrive\\Haccker Earth\\"
train = pd.read_csv(path+"train.csv", sep=',', quotechar='"')
train=train.dropna()

In [4]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(300,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [5]:
%%time
train_tokenized = train.apply(lambda x: w2v_tokenize_text(x['desc']), axis=1).values
train_tokenized_1 = train.apply(lambda x: x['keywords'].split("-"), axis=1).values
train_tokenized_2 = train.apply(lambda x: w2v_tokenize_text(x['name']), axis=1).values


Wall time: 1min 43s


In [6]:
%%time
X_train_desc_average = word_averaging_list(wv,train_tokenized)


2017-06-18 18:46:24,792 : WARNING : cannot compute similarity with no input ['Blurby']
2017-06-18 18:46:24,853 : WARNING : cannot compute similarity with no input ['www.theimpossiblegirl.com']
2017-06-18 18:46:25,309 : WARNING : cannot compute similarity with no input ['AhhhhhhhhhhHh']
2017-06-18 18:46:25,386 : WARNING : cannot compute similarity with no input ['www.foldersnacks.com']
2017-06-18 18:46:25,454 : WARNING : cannot compute similarity with no input []
2017-06-18 18:46:25,547 : WARNING : cannot compute similarity with no input []
2017-06-18 18:46:26,326 : WARNING : cannot compute similarity with no input ['ARELLA']
2017-06-18 18:46:26,831 : WARNING : cannot compute similarity with no input []
2017-06-18 18:46:27,011 : WARNING : cannot compute similarity with no input []
2017-06-18 18:46:27,048 : WARNING : cannot compute similarity with no input ['n/a']
2017-06-18 18:46:27,219 : WARNING : cannot compute similarity with no input []
2017-06-18 18:46:27,365 : WARNING : cannot com

Wall time: 2min 5s


In [ ]:
%%time
X_train_keywords_average = word_averaging_list(wv,train_tokenized_1)


2017-06-18 18:48:48,511 : WARNING : cannot compute similarity with no input ['tweetsourcing']
2017-06-18 18:48:51,093 : WARNING : cannot compute similarity with no input ['droplr']
2017-06-18 18:48:51,812 : WARNING : cannot compute similarity with no input ['colorvision']
2017-06-18 18:48:54,072 : WARNING : cannot compute similarity with no input ['flojugglercom']
2017-06-18 18:48:55,672 : WARNING : cannot compute similarity with no input ['streeteats']
2017-06-18 18:48:55,672 : WARNING : cannot compute similarity with no input ['seamonster3dwebseries']
2017-06-18 18:48:55,904 : WARNING : cannot compute similarity with no input ['antipledge']
2017-06-18 18:48:56,142 : WARNING : cannot compute similarity with no input ['addicube']
2017-06-18 18:48:56,289 : WARNING : cannot compute similarity with no input ['pixelpete', '2010']
2017-06-18 18:48:56,828 : WARNING : cannot compute similarity with no input ['scriptfrenzy', '2010']
2017-06-18 18:48:56,959 : WARNING : cannot compute similarity

In [8]:
%%time
X_train_name_average = word_averaging_list(wv,train_tokenized)

2017-06-18 18:53:25,561 : WARNING : cannot compute similarity with no input ['Blurby']
2017-06-18 18:53:25,592 : WARNING : cannot compute similarity with no input ['www.theimpossiblegirl.com']
2017-06-18 18:53:26,072 : WARNING : cannot compute similarity with no input ['AhhhhhhhhhhHh']
2017-06-18 18:53:26,138 : WARNING : cannot compute similarity with no input ['www.foldersnacks.com']
2017-06-18 18:53:26,207 : WARNING : cannot compute similarity with no input []
2017-06-18 18:53:26,288 : WARNING : cannot compute similarity with no input []
2017-06-18 18:53:26,674 : WARNING : cannot compute similarity with no input ['ARELLA']
2017-06-18 18:53:26,921 : WARNING : cannot compute similarity with no input []
2017-06-18 18:53:27,063 : WARNING : cannot compute similarity with no input []
2017-06-18 18:53:27,096 : WARNING : cannot compute similarity with no input ['n/a']
2017-06-18 18:53:27,210 : WARNING : cannot compute similarity with no input []
2017-06-18 18:53:27,295 : WARNING : cannot com

Wall time: 9.94 s


In [16]:
%%time
df1=pd.DataFrame(X_train_desc_average)
df2=pd.DataFrame(X_train_keywords_average)
df3=pd.DataFrame(X_train_name_average)
df1.columns=["desc_"+str(i) for i in range(300)]
df2.columns=["key_"+str(i) for i in range(300)]
df3.columns=["name_"+str(i) for i in range(300)]
dfFinal=pd.concat([df1, df2,df3], axis=1)

#del train_tokenized
#del train_tokenized_1
#del train_tokenized_2
#del X_train_desc_average
#del X_train_keywords_average
#del X_train_name_average

NameError: name 'X_train_desc_average' is not defined

In [10]:
%%time
del wv

Wall time: 7.95 s


In [15]:
dfFinal=pd.concat([df1, df2,df3], axis=1)
trainFinal=pd.concat([train_1, dfFinal], axis=1)

NameError: name 'df1' is not defined

In [11]:
train['created_date']=train['created_at'].apply(lambda x: pd.to_datetime(x, unit='s'))
train['deadline_date']=train['deadline'].apply(lambda x: pd.to_datetime(x, unit='s'))
train['state_changed_date']=train['state_changed_at'].apply(lambda x: pd.to_datetime(x, unit='s'))
train['created_day']=train['created_date'].apply(lambda x: int(x.day))
train['created_month']=train['created_date'].apply(lambda x: int(x.month))
train['created_year']=train['created_date'].apply(lambda x: int(x.year))
train['created_hour']=train['created_date'].apply(lambda x: int(x.hour))
train['deadline_diff']=(train['deadline_date']-train['created_date']).apply(lambda x: x.total_seconds()/3600/24)
train['state_diff']=(train['state_changed_date']-train['created_date']).apply(lambda x: x.total_seconds()/3600/24)

train_1=train[['project_id','goal','disable_communication','country','currency','backers_count'
               , 'created_day', 'created_month', 'created_year',
       'created_hour', 'deadline_diff', 'state_diff']]

trainFinal=pd.concat([train_1, dfFinal], axis=1)

del train['created_at']
del train['deadline']
del train['state_changed_at']
del train['created_date']
del train['deadline_date']
del train['state_changed_date']
del train['launched_at']

del df1
del df2
del df3
del dfFinal

In [12]:

#Getting Categorical columns
train_obj = trainFinal[['disable_communication','country','currency','created_day' ,'created_month', 'created_year' ,'created_hour']]

train_obj['created_day'] = train_obj['created_day'].astype(object)
train_obj['created_month'] = train_obj['created_month'].astype(object)
train_obj['created_year'] = train_obj['created_year'].astype(object)
train_obj['created_hour'] = train_obj['created_hour'].astype(object)
train_obj_dum = pd.get_dummies(train_obj)



C:\Users\sriram\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\sriram\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\sriram\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

In [19]:
trainFinal.drop(train_obj.columns.values,axis=1,inplace=True)
trainFinal=pd.concat([trainFinal.ix[:,:904], train_obj_dum,train['final_status']], axis=1)


0         1.0
1         0.0
2         0.0
3         1.0
4         0.0
5         0.0
6         0.0
7         1.0
8         0.0
9         0.0
10        0.0
11        0.0
12        1.0
13        1.0
14        0.0
15        1.0
16        0.0
17        1.0
18        0.0
19        1.0
20        1.0
21        1.0
22        1.0
23        1.0
24        1.0
25        1.0
26        0.0
27        1.0
28        1.0
29        1.0
         ... 
108099    0.0
108100    1.0
108101    1.0
108102    0.0
108103    0.0
108104    0.0
108105    0.0
108106    0.0
108107    0.0
108108    0.0
108109    0.0
108110    0.0
108111    0.0
108112    0.0
108113    0.0
108114    0.0
108115    0.0
108116    0.0
108117    0.0
108118    0.0
108119    0.0
108120    0.0
108121    0.0
108122    1.0
108123    1.0
108124    0.0
108125    0.0
108126    1.0
108127    0.0
108128    0.0
Name: final_status, dtype: float64

In [24]:
from utils import *
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss


param_grid = [{
               'clf__n_estimators': [10,15,20,25,50,100],
               'clf__min_samples_leaf':[5,10]}]

pipe1 = Pipeline([('clf', RandomForestClassifier())])


rf = GridSearchCV(pipe1, param_grid, 
                           scoring='accuracy',
                           cv=10, verbose=1,
                           n_jobs=-1)




In [ ]:
rf.fit(trainFinal.ix[:,:1000].values,trainFinal.ix[:,1000])

In [ ]:
1+2